In [ ]:
import numpy as np
import librosa
import scipy.signal as signal
import matplotlib.pyplot as plt

def load_audio(file_path, sr=44100):
    audio, _ = librosa.load(file_path, sr=sr, mono=False)
    if audio.ndim != 2 or audio.shape[0] != 2:
        raise ValueError("Audio must be stereo with shape (2, samples)")
    left = audio[0, :]
    right = audio[1, :]
    return left, right

def calculate_itd(left, right, sr):
    # Compute cross-correlation
    corr = signal.correlate(left, right, mode='full')
    lags = signal.correlation_lags(len(left), len(right), mode='full')
    lag = lags[np.argmax(corr)]
    itd_seconds = lag / sr
    print(f"Estimated ITD: {itd_seconds*1e6:.2f} microseconds")
    return itd_seconds

def calculate_ild(left, right):
    # Calculate RMS energy for each channel
    rms_left = np.sqrt(np.mean(left**2))
    rms_right = np.sqrt(np.mean(right**2))
    ild_db = 20 * np.log10(rms_left / (rms_right + 1e-8))  # avoid divide by zero
    print(f"Estimated ILD: {ild_db:.2f} dB")
    return ild_db

def plot_waveform(left, right, sr):
    time = np.arange(len(left)) / sr
    plt.figure(figsize=(10, 4))
    plt.plot(time, left, label='Left')
    plt.plot(time, right, label='Right', alpha=0.7)
    plt.title('Waveform (Left and Right Channels)')
    plt.xlabel('Time [s]')
    plt.ylabel('Amplitude')
    plt.legend()
    plt.tight_layout()
    plt.show()



In [ ]:
# ---------- MAIN SCRIPT ----------

file_path = 'your_binaural_audio.wav'  # <-- replace with your file
sr = 44100  # sampling rate

# Load audio
left, right = load_audio(file_path, sr)

# Calculate ITD and ILD
itd = calculate_itd(left, right, sr)
ild = calculate_ild(left, right)

# Plot waveform for visual inspection
plot_waveform(left, right, sr)